Based on (Dựa trên: https://www.kaggle.com/code/sudalairajkumar/getting-started-with-text-preprocessing) 

In [2]:
import numpy as np
import pandas as pd
import regex as re
import pandas as pd
import emoji_vietnamese as ev

# Import thư viện pyvi

In [3]:
from pyvi import ViTokenizer, ViPosTagger

# Bảng nguyên âm tiếng Việt và cách mã hóa chúng

In [6]:
bang_nguyen_am = [['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a'],
                  ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
                  ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
                  ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e'],
                  ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
                  ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i'],
                  ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o'],
                  ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
                  ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
                  ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u'],
                  ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
                  ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y']]

bang_ky_tu_dau = ['', 'f', 's', 'r', 'x', 'j']
nguyen_am_to_ids = {}

# Ánh xạ nguyên âm và mã hóa dấu
for i in range(len(bang_nguyen_am)):
    for j in range(len(bang_nguyen_am[i]) - 1):
        nguyen_am_to_ids[bang_nguyen_am[i][j]] = (i, j)

# Chuẩn hóa unicode tổ hợp về dạng NFC
def chuan_hoa_unicode(text):
    text = unicodedata.normalize('NFC', text)
    return text

In [8]:
# Kiểm tra từ có hợp lệ hay không
def is_valid_vietnam_word(word):
    chars = list(word)
    nguyen_am_index = -1
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x != -1:
            if nguyen_am_index == -1:
                nguyen_am_index = index
            else:
                if index - nguyen_am_index != 1:
                    return False
                nguyen_am_index = index
    return True

# Chuẩn hóa dấu cho từ tiếng Việt
# Chuyển câu văn về cách gõ dấu kiểu cũ: dùng òa úy thay oà uý
# Xem tại đây: https://vi.wikipedia.org/wiki/Quy_tắc_đặt_dấu_thanh_trong_chữ_quốc_ngữ
def chuan_hoa_dau_tu_tieng_viet(word):
    if not is_valid_vietnam_word(word):
        return word

    chars = list(word)
    dau_cau = 0
    nguyen_am_index = []
    qu_or_gi = False

    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1:
            continue
        elif x == 9 and index != 0 and chars[index - 1] == 'q':  # Kiểm tra qu
            chars[index] = 'u'
            qu_or_gi = True
        elif x == 5 and index != 0 and chars[index - 1] == 'g':  # Kiểm tra gi
            chars[index] = 'i'
            qu_or_gi = True
        if y != 0:
            dau_cau = y
            chars[index] = bang_nguyen_am[x][0]
        if not qu_or_gi or index != 1:
            nguyen_am_index.append(index)

    if len(nguyen_am_index) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = nguyen_am_to_ids.get(chars[1])
                chars[1] = bang_nguyen_am[x][dau_cau]
            else:
                x, y = nguyen_am_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = bang_nguyen_am[x][dau_cau]
                else:
                    chars[1] = bang_nguyen_am[5][dau_cau] if chars[1] == 'i' else bang_nguyen_am[9][dau_cau]
            return ''.join(chars)
        return word

    for index in nguyen_am_index:
        x, y = nguyen_am_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = bang_nguyen_am[x][dau_cau]
            return ''.join(chars)

    if len(nguyen_am_index) == 2 and nguyen_am_index[-1] == len(chars) - 1:
        x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
        chars[nguyen_am_index[0]] = bang_nguyen_am[x][dau_cau]
    else:
        x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
        chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]

    return ''.join(chars)

# Chuẩn hóa dấu câu
def chuan_hoa_dau_cau_tieng_viet(sentence):
    sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        if len(cw) == 3:
            cw[1] = chuan_hoa_dau_tu_tieng_viet(cw[1])
        words[index] = ''.join(cw)
    return ' '.join(words)

# Tách từ tiếng Việt
# Tách các từ ghép VD : Học sinh học sinh học ⇒ Học_sinh học sinh_học
def tach_tu_tieng_viet(text):
    text = ViTokenizer.tokenize(text)
    return text

# Đưa về chữ viết thường
def chuyen_chu_thuong(text):
    return text.lower()

# Xóa các ký tự thừa, dấu câu, khoảng trắng thừa và chuẩn hóa câu
def chuan_hoa_cau(text):
    text = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text
# Đổi emoji sang text
def doi_emoji_sang_tieng_viet(text):
    return ev.demojize(text)
# Xóa bỏ stopword
stopword = set()
with open('./vietnamese-stopwords.txt', 'r', encoding='utf-8') as f:
    stopword = f.read().split("\n")  # Đọc danh sách stopwords và chia dòng
def xoa_stopwords(text):
    words = []
    for word in text.strip().split():
        if word not in stopword:
            words.append(word)
    return ' '.join(words)
# Xóa địa chỉ email
def xoa_email(text):
    # Sử dụng regex để tìm và xóa email
    email_pattern = r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+'
    return re.sub(email_pattern, '', text)
# Hàm xóa URL
def xoa_url(text):
    # Biểu thức chính quy để nhận diện URL
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    return re.sub(url_pattern, '', text)
# Hàm xử lý tiền xử lý văn bản
def tien_xu_li(text):
    text = chuan_hoa_unicode(text)
    text = xoa_email(text)
    text = xoa_url(text)
    text = chuan_hoa_dau_tu_tieng_viet(text)
    #text = chuan_hoa_dau_cau_tieng_viet(text)
    text = tach_tu_tieng_viet(text)
    text = chuyen_chu_thuong(text)
    text = doi_emoji_sang_tieng_viet(text)
    text = xoa_stopwords(text)
    text = chuan_hoa_cau(text)
    return text
# Hàm chuyển đổi rating thành mảng nhị phân
def convert_rating_to_binary(rating):
    # Bảng mapping từ số sao sang mảng nhị phân
    rating_to_binary = {
        5: [0, 0, 0, 0, 1],
        4: [0, 0, 0, 1, 0],
        3: [0, 0, 1, 0, 0],
        2: [0, 1, 0, 0, 0],
        1: [1, 0, 0, 0, 0]
    }
    return rating_to_binary.get(rating, [0, 0, 0, 0, 0])  # Nếu không có rating thì trả về [0, 0, 0, 0, 0]

# Chương trình chính 

In [9]:
if __name__ == "__main__":
    file_path = './Data.csv'

    # Đọc file CSV
    data = pd.read_csv(file_path)

    # Lấy cột 'CommentText' và xử lý dữ liệu
    commentText = data['CommentText'].fillna('')  # Xử lý NaN bằng chuỗi trống
    commentText_processed = commentText.apply(tien_xu_li)

    data['RatingBinary'] = data['Rating'].apply(convert_rating_to_binary)
    # Xác định vị trí của cột 'Rating' và 'CommentText'
    rating_column_index = data.columns.get_loc('Rating')
    cmt_column_index = data.columns.get_loc('CommentText')
    # Chèn cột 'RatingBinary' và 'prepCommentText'
    data.insert(rating_column_index + 1, 'RatingBinary', data.pop('RatingBinary'))
    data.insert(cmt_column_index + 1, 'prepCommentText', commentText_processed)

    # Lưu file CSV có cột mới
    output_file_path = './ResultData.csv'
    data.to_csv(output_file_path, index=False, encoding='utf-8-sig')

    # Hiển thị kết quả
    print("Ví dụ xử lý : \n")
    print(commentText_processed)
    print("Chuyển đổi rating thành nhị phân:\n")
    print(data[['Rating', 'RatingBinary']].head())

NameError: name 'unicodedata' is not defined